# Chapter 6. 단어에 꼬리표 달기  

시퀀스 레이블링이란 음성, 단어 따위의 시퀀스 데이터에 레이블을 달아 주는 과제를 가리킵니다. 자연어 처리에서 대표적인 시퀀스 레이블링 과제로 개체명 인식이 있습니다. 문장을 토큰화한 뒤 토큰 각각에 인명, 기관명, 장소 등 개체명 태그를 붙여 주는 과제입니다.   
이 장에서는 한국해양대학교에서 공개한 데이터셋에 자체로 제작한 데이터셋을 합친 데이터를 가지고 개체명 인식 모델을 구축하는 방법을 살펴봅니다.

## <6-1 개체명 인식 모델 훑어보기>  

이 장에서는 개체명 인식 과제를 실습합니다. ***개체명 인식(named entity recognition)*** 이란 문장을 토큰화한 뒤 토큰 각각에 인명, 지명, 기관명 등 개체명 태그를 붙여 주는 과제입니다.  
예를 들어 다음과 같습니다.

- 그 제품 삼성 건가요(입력 문장)  
$\rightarrow$ `그: 개체명 아님` `제품: 개체명 아님` `삼성: 기관명` `건가요: 개체명 아님` (분석 결과)

이번 실습에서 사용할 데이터는 한국해양대학교 자연언어처리연구실에서 공개한 데이터를 사용합니다. 여기에 자체적으로 제작한 데이터를 추가로 포함했습니다. 자체 제작 데이터는 윤주성 님이 개발한 BERT 기반 개체명 인식 모델로 초벌 레이블링을 수행한 뒤 수작업으로 해당 레이블이 맞는지 검토해 만들었습니다.

개체명 태그 수와 종류는 데이터를 제작할 때 자유롭게 정할 수 있습니다. 하지만 이번 실습은 한국해양대학교 데이터셋을 기본으로 하므로 이 데이터의 태그 체계를 따릅니다. 그 종류는 다음과 같습니다. 총 10개 태그입니다.

|태그|종류|태그|종류|
|:---:|:---:|:---:|:---:|
|PER|인명|DUR|기간|
|LOC|지명|MNY|통화|
|ORG|기관명|PNT|비율|
|DAT|날짜|NOH|기타 수량표현|
|TIM|시간|POH|기타|

우리가 만들 개체명 인식 모델의 입력은 토큰 시퀀스 입니다. 앞에서 든 예를 기준으로 설명한다면 모델의 입력은 `[그, 제품, 삼성, 건가요]`가 됩니다. 이 모델의 출력은 각 토큰이 어떤 개체명 태그에 속할지 확률을 나타냅니다. 다음과 같습니다.

> **[인명(PER)일 확률, 지명(LOC)일 확률, 기관명(ORG)일 확률, 날짜(DAT)일 확률, 시간(TIM)일 확률, 기간(DUR)일 확률, 통화(MNY)일 확률, 비율(PNT)일 확률, 기타 수량표현(NOH)일 확률, 기타(POH)일 확률, 어떤 개체명도 아닐 확률]**

예를 들어 입력된 토큰이 5개라면 모델은 5개 토큰 각각에 해당 토큰이 어떤 개체명에 속할지 확률값을 내어 줍니다. 이를 적당한 후처리 과정을 거쳐 사람이 보기에 좋은 형태로 가공해 주면 문장 내에서 어떤 부분이 개체명에 속하는지 알 수 있습니다.

### 모델 구조
---
이 책에서 사용하는 개체명 인식 모델은 다음 그림과 같은 구조입니다. 입력 문장을 토큰화한 뒤 문장 시작과 끝을 알리는 스페셜 토큰 `CLS`와 `SEP`를 각각 원래 토큰 시퀀스 앞뒤에 붙입니다. 이를 BERT 모델에 입력하고 모든 토큰에 대해 BERT 모델 마지막 레이어의 출력을 뽑습니다. 이들 토큰 벡터 각각에 작은 추가 모듈을 덧붙여 모델의 출력이 해당 토큰이 특정 개체명에 속할 확률이 되도록 합니다.

<center><그림 6-1 개체명 인식></center>

<p align="center"><img src="https://i.imgur.com/MjbcakT.png">  

<center>출처 : ratsgo's NLPBOOK</center>

### 태스크 모듈
---
개체명 인식 모듈에 붙는 태스크 모듈의 구조는 다음 그림과 같습니다.  
우선 마지막 레이어의 개별 토큰 벡터(그림에서 $x$)에 드롭아웃을 적용합니다. 그다음 개별 토큰 벡터 각각에 가중치 행렬을 곱해 개별 토큰 벡터 각각을 분류해야 할 범주 수만큼의 차원을 갖는 벡터로 변환합니다(그림에서 $h$). 만일 토큰 벡터 하나가 768차원이고 분류 대상 범주 수가 11개(개체명 태그 10개 범주 + '개체명 아님' 범주)라면 가중치 행렬 크기는 768 $\times$ 11이 됩니다. 여기에 소프트맥스 함수를 취하면 모델의 최종 출력(그림에서 $y$)이 됩니다. 

<center><그림 6-2 개체명 인식 태스크 모듈 (1)></center>

<p align="center"><img src="https://i.imgur.com/tDKNUJh.png">  

<center>출처 : ratsgo's NLPBOOK</center>

다만 개체명 인식 모델은 토큰 각각을 개체명 확률로 내어 주므로 개별 토큰 벡터 차원 수가 768, 분류 대상 개체명 범주 수가 $m$개, 입력 토큰 수가 $n$개라면 실제로는 다음 그림과 같은 구조가 됩니다. 이렇게 만든 모델의 최종 출력과 정답 레이블을 비교해 모델 출력이 정답 레이블과 최대한 같아지도록 BERT 레이어 전체를 포함한 모델 전체를 업데이트합니다.

<center><그림 6-3 개체명 인식 태스크 모듈 (2)></center>

<p align="center"><img src="https://i.imgur.com/U5mPOBh.png">  

<center>출처 : ratsgo's NLPBOOK</center>